In [1]:
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader, NeighborLoader
from torch.utils.data import random_split

In [2]:
from model import SimpleGAT, BenchmarkGAT
from train import train_classifier

In [3]:
import torch_geometric
print(torch_geometric.typing.WITH_PYG_LIB)

True


# Performance on Planetoid datasets

In [4]:
train_dataset = Planetoid(root="Planetoid/PubMed/", name="PubMed", split="full", num_val=500)

In [5]:
train_loader = NeighborLoader(train_dataset[0], input_nodes=train_dataset[0].train_mask,
                              num_neighbors=[50, 50, 50], shuffle=True, batch_size=64)
val_loader =NeighborLoader(train_dataset[0], input_nodes=train_dataset[0].val_mask,
                              num_neighbors=[50, 50], shuffle=True, batch_size=20)

loaders = {"train": train_loader, "val": val_loader}

In [6]:
model = SimpleGAT(in_channels=500, out_channels = 3 ,heads=4, hidden_size=100).to("cuda")

In [7]:
#test pass trough model
d = next(iter(train_loader))
d

Data(x=[7581, 500], edge_index=[2, 20504], y=[7581], train_mask=[7581], val_mask=[7581], test_mask=[7581], n_id=[7581], e_id=[20504], num_sampled_nodes=[4], num_sampled_edges=[3], input_id=[64], batch_size=64)

In [8]:
o = model(d.to("cuda"))
o.shape

torch.Size([7581, 3])

In [9]:
loss_fn = torch.nn.CrossEntropyLoss()

In [10]:
optim = torch.optim.Adadelta(model.parameters(), lr=1e-2)

In [ ]:
train_classifier(model, optim, loaders, loss_fn, 100, print_every=10, test_every=5)

Epoch 0, Train Loss: 1.08
Epoch 0, Accuracy: 45.36%


## Compare to baseline GAT model

In [ ]:
model = BenchmarkGAT(in_channels=500, out_channels = 3 ,heads=4, hidden_size=100).to("cuda")

In [ ]:
optim = torch.optim.Adam(model.parameters(), lr=1e-2)

In [ ]:
train_classifier(model, optim, loaders, loss_fn, 100, print_every=10, test_every=5)